In [2]:
#import libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import random as rnd

# read data into a DataFrame
data = pd.read_csv('/Users/kaushikshivakumar/Downloads/filteredallcancerdataforuse.csv', index_col=0)
print data.shape
x = list(data.columns.values)
data

(1029, 74)


,State,County,YearsofPotentialLifeLostRate,PercentFairOrPoor,PhysicallyUnhealthyDays,MentallyUnhealthyDays,PercentLowBirthWeight,PercentSmokers,PercentObese,FoodEnvironmentIndex,...,PercentNativeHawaiianOrOtherPacificIslander,PercentHispanic,PercentNonHispanicWhite,PercentNotProficientinEnglish,PercentFemale,PercentRural,PercentInsufficientSleep,Incidences,IncidencesPTTP,Cutoff
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,9215,19,4.4,4.3,9,19,31,7.1,...,0.1,2.7,75.599998,0.5,51.400002,42.000000,38,265,47.838253,1
1003,Alabama,Baldwin,7455,16,4.1,4.2,9,19,27,7.6,...,0.1,4.6,83.000000,1.4,51.200001,42.299999,33,1093,54.619686,1
1009,Alabama,Blount,8656,21,4.6,4.5,7,19,32,8.3,...,0.1,8.7,87.800003,1.8,50.500000,90.000000,36,287,49.723661,1
1015,Alabama,Calhoun,10352,22,4.9,4.9,9,24,32,6.2,...,0.1,3.5,72.900002,0.8,51.799999,33.700001,39,657,56.678974,1
1031,Alabama,Coffee,7296,21,4.8,4.6,9,19,30,6.9,...,0.4,6.4,71.300003,2.2,50.500000,47.200001,35,257,50.482233,1
1033,Alabama,Colbert,10076,21,4.8,4.8,10,19,37,7.0,...,0.1,2.6,78.800003,0.4,51.799999,43.900002,36,328,60.136039,1
1043,Alabama,Cullman,9491,20,4.5,4.7,9,20,35,7.7,...,0.0,4.4,92.099998,1.0,50.500000,73.199997,35,437,53.758811,1
1045,Alabama,Dale,8368,21,4.5,4.6,8,20,36,6.3,...,0.1,6.1,70.199997,0.7,50.500000,50.900002,41,250,50.521381,1
1049,Alabama,DeKalb,9813,23,4.8,4.7,9,19,31,7.9,...,0.7,14.2,80.599998,4.5,50.700001,90.099998,36,317,44.607050,1


In [3]:
#find which columns have the strongest correlations
thresh = 0.3
importantCols = []
valone = 0
valtwo = 1
count = 0
for i in range (3, 72):
    newFrame = data[['IncidencesPTTP', list(data.columns.values)[i]]]
    currentVal = (newFrame.corr('pearson')).iloc[0, 1]
    if (currentVal > thresh or currentVal < -thresh):
        importantCols.append(list(data.columns.values)[i])
        #importantCols.append(currentVal)
        count = count+1
print count
importantCols

10


['AssociationRate',
 'InjuryDeathRate',
 'PercentDiabetic',
 'DrugOverdoseMortalityRate',
 'PercentLessThan18',
 'Percent65andover',
 'PercentHispanic',
 'PercentNonHispanicWhite',
 'PercentNotProficientinEnglish',
 'PercentRural']

In [4]:
#filter a new dataframe down to these columns
importantData = data[['AssociationRate',
 'InjuryDeathRate',
 'PercentDiabetic',
 'DrugOverdoseMortalityRate',
 'PercentLessThan18',
 'Percent65andover',
 'PercentHispanic',
 'PercentNonHispanicWhite',
 'PercentNotProficientinEnglish',
 'PercentRural', 'IncidencesPTTP']]
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
importantData.shape

(1029, 11)

In [5]:
#normalize the data
for i in range (0, 10):
    print i
    avg = 0;
    csum = 0;
    ccount = 0;
    cavg = 0;
    cboolarr = importantData.isnull()
    for j in range (0, 1029):
        cval = importantData.iloc[j, i]
        cbool = cboolarr.iloc[j, i]
        if (not cbool):
            csum = csum + int(cval)
            ccount = ccount + 1
    cavg = csum/ccount
    for j in range (0, 1029):
        if (not cbool):
            importantData.iloc[j, i] = (importantData.iloc[j, i])/cavg

0


/Users/kaushikshivakumar/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1
2


/Users/kaushikshivakumar/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/kaushikshivakumar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3
4
5
6
7
8
9


In [6]:
lm = smf.ols(formula='IncidencesPTTP ~ AssociationRate + InjuryDeathRate + PercentDiabetic + DrugOverdoseMortalityRate + PercentLessThan18 + Percent65andover + PercentHispanic + PercentNonHispanicWhite + PercentNotProficientinEnglish + PercentRural', data = importantData).fit();
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         IncidencesPTTP   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     367.2
Date:                Tue, 28 Mar 2017   Prob (F-statistic):               0.00
Time:                        20:32:24   Log-Likelihood:                -3038.0
No. Observations:                1005   AIC:                             6098.
Df Residuals:                     994   BIC:                             6152.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------
Intercept                        13.2100      2.530      5.221      0.000         8.244    18.176
AssociationRate                   3.5796      0.576      6.215      0.000         2.449     4.710
InjuryDeathRate                  -4.3493      0.935     -4.650      0.000        -6.185    -2.514
PercentDiabetic                   8.5223      1.441      5.915      0.000         5.695    11.349
DrugOverdoseMortalityRate         3.4711      0.461      7.532      0.000         2.567     4.375
PercentLessThan18                -4.8292      1.702     -2.838      0.005        -8.168    -1.490
Percent65andover                 24.7830      0.990     25.035      0.000        22.840    26.726
PercentHispanic                  -1.6134      0.275     -5.872      0.000        -2.153    -1.074
PercentNonHispanicWhite           4.9766      1.342      3.707      0.000         2.342     7.611
PercentNotProficientinEnglish     0.5371      0.234      2.299      0.022         0.079     0.996
PercentRural                     -0.4248      0.301     -1.412      0.158        -1.015     0.166
==============================================================================
Omnibus:                       51.616   Durbin-Watson:                   1.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.989
Skew:                          -0.334   Prob(JB):                     2.62e-23
Kurtosis:                       4.427   Cond. No.                         65.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
#randomly create two dataframes, one for testing, one for predicting
importantTrainingData = importantData.copy(deep = False)
importantPredictingData = importantData.copy(deep = False)

for i in range (1028, -1, -1):
    if (rnd.randint(0, 3) == 0):
        importantTrainingData.drop(importantTrainingData.index[[i]], inplace=True)
    else:
        importantPredictingData.drop(importantPredictingData.index[[i]], inplace=True)
print importantTrainingData.shape
print importantPredictingData.shape

#create the model and assess statistical significance
lm = smf.ols(formula='IncidencesPTTP ~ AssociationRate + InjuryDeathRate + PercentDiabetic + DrugOverdoseMortalityRate + PercentLessThan18 + Percent65andover + PercentHispanic + PercentNonHispanicWhite + PercentNotProficientinEnglish + PercentRural', data = importantData).fit();
lm.summary()

(761, 11)
(268, 11)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         IncidencesPTTP   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     367.2
Date:                Tue, 28 Mar 2017   Prob (F-statistic):               0.00
Time:                        20:58:17   Log-Likelihood:                -3038.0
No. Observations:                1005   AIC:                             6098.
Df Residuals:                     994   BIC:                             6152.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------
Intercept                        13.2100      2.530      5.221      0.000         8.244    18.176
AssociationRate                   3.5796      0.576      6.215      0.000         2.449     4.710
InjuryDeathRate                  -4.3493      0.935     -4.650      0.000        -6.185    -2.514
PercentDiabetic                   8.5223      1.441      5.915      0.000         5.695    11.349
DrugOverdoseMortalityRate         3.4711      0.461      7.532      0.000         2.567     4.375
PercentLessThan18                -4.8292      1.702     -2.838      0.005        -8.168    -1.490
Percent65andover                 24.7830      0.990     25.035      0.000        22.840    26.726
PercentHispanic                  -1.6134      0.275     -5.872      0.000        -2.153    -1.074
PercentNonHispanicWhite           4.9766      1.342      3.707      0.000         2.342     7.611
PercentNotProficientinEnglish     0.5371      0.234      2.299      0.022         0.079     0.996
PercentRural                     -0.4248      0.301     -1.412      0.158        -1.015     0.166
==============================================================================
Omnibus:                       51.616   Durbin-Watson:                   1.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.989
Skew:                          -0.334   Prob(JB):                     2.62e-23
Kurtosis:                       4.427   Cond. No.                         65.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#assess percent error of model vs percent error when using the average cancer incidences as prediction
#as seen below, using average for prediction results in over 3 times the error as using the model
#importantPredictingData = importantData.copy(deep=False)
importantPredictingData.replace('', np.nan, inplace=True)
importantPredictingData = importantPredictingData.dropna()
print importantPredictingData.shape
asum = 0
avg = 0
for i in range (0, len(importantPredictingData.index)):
    asum = asum + importantPredictingData.iloc[i, 10]
avg = asum/len(importantPredictingData.index)
sum1 = 0
sum2 = 0

#print importantPredictingData
for i in range(0, len(importantPredictingData.index)):
    X_new = importantPredictingData.iloc[i:i+1]
    X_new = pd.DataFrame(X_new)
    answer = X_new['IncidencesPTTP'].iloc[0]
    del X_new['IncidencesPTTP']
    coeffs = pd.DataFrame(lm.params)
    intercept = coeffs.iloc[0, 0]
    #print intercept
    coeffs = coeffs.drop(coeffs.index[0])
    #print X_new
    #print coeffs
    #print X_new.shape;
    #print coeffs.shape;
    #print intercept;
    #print X_new.dot(coeffs)
    error1 = 100*abs(X_new.dot(coeffs).iloc[0, 0] + intercept - answer)/answer
    error2 = 100*abs(avg - answer)/answer
    #print error
    sum1 = sum1 + error1
    sum2 = sum2 + error2
    #print error
#print sum
print str(sum1/len(importantPredictingData.index)) + " " + str(sum2/len(importantPredictingData.index))

(264, 11)
7.70941521441 19.4895225684
